# Information Extraction
This notebook uses the regex information extraction from the api.

In [1]:
import os
import pandas as pd
import numpy as np
import random
import json
import pickle
import re
from processer import Processer
from api import get_supplier, extract_data
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from gensim.sklearn_api.d2vmodel import D2VTransformer

In [2]:
train = pd.read_csv('train.txt')
train = train.set_index('nummer')
X = train["text"]
Y = train["lief2"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [3]:
train.head()

,Unnamed: 0,Unnamed: 0.1,sumbrutto,sumnetto,lief2,text
nummer,,,,,,
390486,4,4,0.00,0.00,"(158,1,0)",Yvonne Burri\nae ET eS eS a i a ee ee\n\nVon: ...
800045,254,254,350.00,350.00,"(158,1,0)",. SOIOUS\n« SOMAT\n\nMALA’ YSIA\n\n \n\ndriven...
800098,279,279,258.18,258.18,"(158,1,0)",\n\n \n\na a cS\n“BOMATEC\n: MALAYSIA\ndriven...
800099,278,278,126.00,126.00,"(158,1,0)",_ KIOOTN\nTEC:\n\n \n\n \n\nMALAYSIA\ndriven b...
800100,277,277,284.00,284.00,"(158,1,0)",_ .BOMATEC SCOOP\n\nMALAYSIA\n\ndriven by pass...


In [4]:
c = pickle.load(open('model.pkl', 'rb'))
pred = c.predict(X)
pred_proba = c.predict_proba(X)
print('F1-score:', f1_score(Y, pred, average='weighted'))

F1-score: 0.9219213552090685


In [5]:
idx = random.randint(0, len(Y))
print('Document no: %d' % Y.index[idx])
print('Predicted class: %s' % json.loads(get_supplier(pred[idx]))['name'])
print('Probability: %s' % np.amax(pred_proba[idx]))
print('True class: %s' % json.loads(get_supplier(Y.iloc[idx]))['name'])

Document no: 806621
Predicted class: Bomatec Management AG
Probability: 0.9462852550596345
True class: Bomatec Management AG


In [14]:
print(X.loc[Y.index[idx]])

3OMATEC:

more than just magnets

Bomatec Management AG
Hofstrasse 1

806674

 

 

 

CH-8181 Hori
Bomatec Automotive AG
Hofstrasse 1
8181 HORI
Rechnung 1-400088 Seite 1/3
Dokumentendatum 28.09.2018 ihr Ansprechpartner Ivo Boni
Kunden Nr. 20 Ihr Zeichen
Unser Zeichen mha Vom
Zustandig Armin Sejfic EORI Nummer
Telefon direkt +41 44 872 10 46
E-Mail armin.sejfic@ bomatec.ch
MARQUARDT MAGNET HAPTIK
Das Leistungsdatum entspricht dem Rechnungsdatum.
Pos Artikel Menge Preis/Einh. % Gesamtpreis
CHF CHF
Auftragsbestétigung 1-200055 vom 27.07.2018
Marquardt Magnet Haptik 46h 120.00/1 5'520.00

Engineering Services BACH

Armin Sejfic , effektive Arbeitszeit 1.00 h
27.07.2018
Sitzungsvorbereitung und Konzeptreview

Armin Sejfic , effektive Arbeitszeit 2.00 h
27.07.2018
Sitzung Marquardt

Armin Sejfic , effektive Arbeitszeit 1.00 h
27.07.2018
Besuchsbericht schreiben

AGB / Technische Lieferbedingungen siehe www.bomatec.ch

 

 

Bomatec Management AG Tel; +41 44872 1000
Hofstrasse 1 Fax: +41 44 

In [38]:
folder_path = "./data/PI20_" + str(Y.index[idx])
file_name = os.listdir(os.getcwd() + folder_path)[0]
os.chdir(folder_path)
os.startfile(file_name)
os.chdir('../..')

In [12]:
regex_template, text, data = extract_data(pred[idx], X.iloc[idx])
print(pd.DataFrame.from_dict(data, orient='index'))

                         0
invoice_number    1-400088
amount            5'945.04
invoice_date    28.09.2018
currency               CHF


In [9]:
train.iloc[idx]

Unnamed: 0                                                    328
Unnamed: 0.1                                                  328
sumbrutto                                                 5945.04
sumnetto                                                     5520
lief2                                                  (1051,1,0)
text            3OMATEC:\n\nmore than just magnets\n\nBomatec ...
Name: 806621, dtype: object